In [13]:
import pika
import uuid

class QuoteRpcClient(object):# a connection, channel and declare an exclusive 'callback' queue for replies
    def __init__(self):
        self.connection = pika.BlockingConnection(pika.ConnectionParameters(host='localhost'))

        self.channel = self.connection.channel()

        result = self.channel.queue_declare(exclusive=True)
        self.callback_queue = result.method.queue

        self.channel.basic_consume(self.on_response, no_ack=True,
                                   queue=self.callback_queue)

    def on_response(self, ch, method, props, body):
        if self.corr_id == props.correlation_id:
            self.response = body

    def call(self, n):
        self.response = None
        self.corr_id = str(uuid.uuid4())
        self.channel.basic_publish(exchange='',
                                   routing_key='Pricing',
                                   properties=pika.BasicProperties(
                                         reply_to = self.callback_queue,
                                         correlation_id = self.corr_id,
                                         ),
                                   body=str(n))
        while self.response is None:
            self.connection.process_data_events()
        return str(self.response)

quote_rpc = QuoteRpcClient()

s = str(input("enter the name of part for which price is needed\n"))
print(" Requesting price of item-", s)
response = quote_rpc.call(ord(s))
print("Got %r " % response)

'''Here a chennel and a connection is established to connect to the server, and a exclusive 'callback' queue for 
servers to reply in them this queue is unique for each client irrespective of the queue for server which is comman for all.
Then the 'on_response' callback executed on every response messagechecks if the correlation_id is the one we're looking for. 
If so, it saves the response in self.response and breaks the consuming loop.The call method first we generate a unique 
correlation_id number and save it as the 'on_response' callback function will use this value to catch the appropriate response.
And finally the response is returned back to the user'''

enter the name of part for which quote is needed
f
 Requesting price for f
Got "b'Requested Part is unavailable'" 
